# This is a work-in-progress notebook

We wish to know this:

1. How well does the model identify the correct number of senses for the target word?
2. **How well does the model identify the correct senses for the target word?**
3. **How well does the model assign the right words to a given sense of the target word?**
4. How well does the model assign the senses to the time intervals for the target word?

The script will evaluate **Q2** and **Q3**. Q4 will follow.

In [103]:
# Basic variables and imports:

import codecs, csv, os, time, re, io
from os import listdir
from os.path import isfile, join

# directories
dir_in = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "evaluation", "evaluation_input"))
dir_out = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "evaluation", "evaluation_output"))


#s_senses = io.open(dir_in+"/senses_69419.txt","r")
#k_senses = io.open(dir_in+"/mus.dat","r")

# DEBUG:
s_senses = io.open(dir_in+"/senses_69419_debug.txt","r")
k_senses = io.open(dir_in+"/mus_debug.dat","r")
# k0 = mus4
# k1 = mus3
# k2 = mus2
# k3 = mus1
# k4 = nothing

file_senses = s_senses.readlines()[1:]
output_senses = k_senses.read()

## TODO

- ~~create the notebook~~
- ~~organise the notebook~~
- ~~write "general idea" pseudocode for the evaluation~~
- ~~get input files~~
- ~~figure out data structures to store the variables~~
- write actual code

## Q2: How well does the model identify the correct senses for the target word?

### Pseudocode 

In [2]:
# For each target word, we have a list of senses  s (given by the expert)
# For each target word, we have a list of senses k (given by the model)
# This Q consists in matching s and k, and doing so in a confident way --> confidence score

for each k:
    for each s:
        create conf(k,s)

# What is conf(k,s)?
        conf(k,s) = (p1*match(w1,s)+p2*match(w1,s)+px(wx,s))/10 WHERE
    
            px = probability of word wx 
                
                and
            
            match(wx,s) =   1/number_of_senses_assigned_to_wx if s_is_one_of_them 
            
                    or 
                            0 if w_is_not_associated_to_s
                
# Once we have gone through all s for one k, we have to choose the best k for s. How? (TBD, cfr Valerio and Barbara)

# Once all ks have been assigned to all ss (or NA), we can calculate a general confidence score for the model.
# One easy way to do that: 

conf_score_model = number_of_non_NA/k

SyntaxError: invalid syntax (<ipython-input-2-82fa54601c9a>, line 5)

### Real code

Steps:

- extract all senses from the file
- use those senses as keys for a dictionary, `dict_of_words`
- fill the dictionary: for each key, we store a list of words pertaining to that sense
- transform the lists as sets so as to remove duplicates within the same sense
- create a dictionary with a word as a key and its weight as a value, depending on how many senses it appears
- parse the model output and get the probability weights for each word
- do not take into account the first line
- take care of empty lines

Todo:



In [104]:
expert_senses = list() # list where we store all sense ids provided by expert
for s in file_senses: # 60 for testing purposes
    s = s.split("\t")
    sense = s[11] # The sense ID is after the 10th tab
    expert_senses.append(sense)
    
#print(len(expert_senses),expert_senses,len(set(expert_senses)))

expert_senses = list(set(expert_senses)) # we only keep the unique senses
number_of_s = len(expert_senses)  # we create a variable that stores the number of unique senses
print("Number of senses:",number_of_s)

# This dictionary has a sense as a key, and a list of words as a value. 
dict_of_words = dict()
# This list stores all words
list_of_all_words = list()
# This dictionary stores all words as keys and their weight as value
word_weight = dict()

for i in range(0,number_of_s): # for each sense, we create a dictionary entry which has a list as value
    dict_of_words[expert_senses[i]] = list()

    for s in file_senses: # we go back in the file
        s = s.split("\t") # splitting on tabs
        
        sentence_of_ids = s[8] # 8 is for IDs, 9 is for words
        list_of_ids = sentence_of_ids.split(" ")  # splitting on spaces
        for word_id in list_of_ids:
            if s[11] == expert_senses[i]:      # we store all words for one sense 
                dict_of_words[expert_senses[i]].append(word_id)
            list_of_all_words.append(word_id) # we store all words, we'll iterate over that for scores
        

    # Here, we remove duplicates
    #dict_of_words[expert_senses[i]].append("79223") #testing
    dict_of_words[expert_senses[i]] = set(dict_of_words[expert_senses[i]]) 
      
    print(expert_senses[i],set(dict_of_words[expert_senses[i]]))
    print("\n\n")


Number of senses: 4
mus-1 {'mus1h', 'mus1e', 'mus1l', 'mus1n', 'mus1m', 'mus1p', 'mus1q', 'mus1d', 'mus1c', 'mus1k', 'mus1o', 'mus1a', 'mus1g', 'mus1f', 'mus1i', 'mus1j', 'mus1b'}



mus-4 {'38472', 'mus4b', '101851', '6898', '66761', '75005', '13942', '13813', '104311', '63873', 'mus4c', '13315', 'mus4e', 'mus4a', '73672', 'mus4d'}



mus-3 {'mus3j', 'mus3p', 'mus3f', 'mus3o', 'mus3q', 'mus3e', 'mus3l', 'mus3m', 'mus3b', 'mus3c', 'mus3i', 'mus3n', 'mus3h', 'mus3d', 'mus3a', 'mus3k', 'mus3g'}



mus-2 {'mus2h', 'mus2g', 'mus2i', 'mus2p', 'mus2j', 'mus2f', 'mus2m', 'mus2d', 'mus2b', 'mus2o', 'mus2n', 'mus2l', 'mus2q', 'mus2k', 'mus2e', 'mus2c', 'mus2a'}





In [105]:
# For every word in the list of words that we have
# we count the number of senses it appears in
# we use that number to divide its importance: 1 sense = 1 importance; 2 senses = 0.5 importance
# this can be finetuned

for word in list_of_all_words:
    x = 0
    for i in range(0,number_of_s):
        if word in dict_of_words[expert_senses[i]]:
            x += 1 
        if x != 0:
            word_weight[word] = 1/x
        
    
    print(word,word_weight[word])  

38472 1.0
75005 1.0
63873 1.0
101851 1.0
13315 1.0
13942 1.0
13813 1.0
66761 1.0
6898 1.0
104311 1.0
73672 1.0
mus4a 1.0
mus4b 1.0
mus4c 1.0
mus4c 1.0
mus4d 1.0
mus4e 1.0
mus3a 1.0
mus3b 1.0
mus3c 1.0
mus3d 1.0
mus3e 1.0
mus3f 1.0
mus3g 1.0
mus3h 1.0
mus3i 1.0
mus3j 1.0
mus3k 1.0
mus3l 1.0
mus3m 1.0
mus3n 1.0
mus3o 1.0
mus3p 1.0
mus3q 1.0
mus2a 1.0
mus2b 1.0
mus2c 1.0
mus2d 1.0
mus2e 1.0
mus2f 1.0
mus2g 1.0
mus2h 1.0
mus2i 1.0
mus2j 1.0
mus2k 1.0
mus2l 1.0
mus2m 1.0
mus2n 1.0
mus2o 1.0
mus2p 1.0
mus2q 1.0
mus1a 1.0
mus1b 1.0
mus1c 1.0
mus1d 1.0
mus1e 1.0
mus1f 1.0
mus1g 1.0
mus1h 1.0
mus1i 1.0
mus1j 1.0
mus1k 1.0
mus1l 1.0
mus1m 1.0
mus1n 1.0
mus1o 1.0
mus1p 1.0
mus1q 1.0
38472 1.0
75005 1.0
63873 1.0
101851 1.0
13315 1.0
13942 1.0
13813 1.0
66761 1.0
6898 1.0
104311 1.0
73672 1.0
mus4a 1.0
mus4b 1.0
mus4c 1.0
mus4c 1.0
mus4d 1.0
mus4e 1.0
mus3a 1.0
mus3b 1.0
mus3c 1.0
mus3d 1.0
mus3e 1.0
mus3f 1.0
mus3g 1.0
mus3h 1.0
mus3i 1.0
mus3j 1.0
mus3k 1.0
mus3l 1.0
mus3m 1.0
mus3n 1.0
mus3o 1.

#### parsing output.dat
- split on "===============  per time  ===============" and keep first part
- transform that into a list, then
- get lines that start with "p(w|s)"
- count those, k = that number
- split the line on ":", keep the second part
- split the rest on ";", it's [ID] = prob_from_this_ID

In [106]:
lines_output = output_senses.split("===============  per time  ===============")[0].split("\n")

number_of_the_k = 0

k_words_with_prob = dict()

for line in lines_output:
    if line[:6] == "p(w|s)":
        line = line.split(":")[1]
        line = line.split(";")
        print(number_of_the_k,line)
        dico_word_prob = dict()
        k_words_with_prob[number_of_the_k] = list()
        
        line = line[:-1] # last item of the list is empty
        for word_prob in line:
        
            word_prob = word_prob.split(",")
            for word in word_prob:
                matchObj = re.findall("([\d.\w]*)",word)
                if matchObj:
                    
                    matchObj = list(filter(None,matchObj))
                    
            
            print("word id",matchObj[0],"; probability",matchObj[1])
            dico_word_prob[matchObj[0]] = matchObj[1]
        k_words_with_prob[number_of_the_k] = dico_word_prob

        number_of_the_k += 1
        



0 ['  38472 (1) ', ' 75005 (1) ', ' 63873 (1) ', ' 101851 (1) ', ' 13315 (1) ', ' 13942 (1) ', ' 13813 (1) ', ' 66761 (1) ', ' 6898 (1) ', ' 73672 (1) ', ' ']
word id 38472 ; probability 1
word id 75005 ; probability 1
word id 63873 ; probability 1
word id 101851 ; probability 1
word id 13315 ; probability 1
word id 13942 ; probability 1
word id 13813 ; probability 1
word id 66761 ; probability 1
word id 6898 ; probability 1
word id 73672 ; probability 1
1 ['  mus3a (1) ', ' mus3b (1) ', ' mus3c (1) ', ' mus3d (1) ', ' mus3e (1) ', ' mus3f (1) ', ' mus3g (1) ', ' mus3h (1) ', ' mus3i (1) ', ' mus3j (1) ', ' ']
word id mus3a ; probability 1
word id mus3b ; probability 1
word id mus3c ; probability 1
word id mus3d ; probability 1
word id mus3e ; probability 1
word id mus3f ; probability 1
word id mus3g ; probability 1
word id mus3h ; probability 1
word id mus3i ; probability 1
word id mus3j ; probability 1
2 ['  mus2a (1) ', ' mus2b (1) ', ' mus2c (1) ', ' mus2d (1) ', ' mus2e (1) ', ' m

#### k_words_with_prob
This dictionary has the sense number 'k' as keys and the a dictionary of [word] = probability as values.
Example below.

In [107]:
print("Probability for word ID 5390 in sense k = 4:",k_words_with_prob[4]["5390"])

Probability for word ID 5390 in sense k = 4: 0.019


In [108]:
for key in k_words_with_prob.keys():
    print("output sense",key)
    for i in range(0,number_of_s):
        print("\texpert sense number ", i, expert_senses[i])
        for second_key in k_words_with_prob[key].keys(): # Barbara's note: shouldn't it be k_words_with_prob[i] here?
            print("\t\tword from annotation for sense", key, ":", second_key)
            if second_key in dict_of_words[expert_senses[i]]:
                print("\t\t\tword ", second_key, "is in output for sense", key, "with probability:", k_words_with_prob[key][second_key], "and weight:", word_weight[second_key])

                
# Here we get all the senses and for each sense we do a matching between the k words and s words and get the probability
# For some reason the first word for each sense arrives several times


output sense 0
	expert sense number  0 mus-1
		word from annotation for sense 0 : 38472
		word from annotation for sense 0 : 75005
		word from annotation for sense 0 : 63873
		word from annotation for sense 0 : 101851
		word from annotation for sense 0 : 13315
		word from annotation for sense 0 : 13942
		word from annotation for sense 0 : 13813
		word from annotation for sense 0 : 66761
		word from annotation for sense 0 : 6898
		word from annotation for sense 0 : 73672
	expert sense number  1 mus-4
		word from annotation for sense 0 : 38472
			word  38472 is in output for sense 0 with probability: 1 and weight: 1.0
		word from annotation for sense 0 : 75005
			word  75005 is in output for sense 0 with probability: 1 and weight: 1.0
		word from annotation for sense 0 : 63873
			word  63873 is in output for sense 0 with probability: 1 and weight: 1.0
		word from annotation for sense 0 : 101851
			word  101851 is in output for sense 0 with probability: 1 and weight: 1.0
		word from annot

In [109]:
## Calculating confidence score for each (words_of_k,words_of_s) pair

# conf(k,s) = (p1*match(w1,s)+p2*match(w1,s)+px(wx,s))/10
        # match(wx,s) =   1/number_of_senses_assigned_to_wx if s_is_one_of_them 

##### TODO: for now conf[k,s] is multiplied by the number of expert senses --- FIX
    
print("number of expert senses s:",number_of_s)
print("number of model output senses k:",len(k_words_with_prob.keys()))
compteur = 0

match = dict()
conf = dict()
for k in k_words_with_prob.keys():  # for each output sense, we go through...
    print("\n")
    print("Choose best match for k =",k)
    for s in range(0,number_of_s):       # each expert sense
        
        conf[k,s] = 0 
        
        #print("expert sense",s)
        for mot in k_words_with_prob[k]:      # for each word within output by the model for the output sense
            #print(k,mot)
            #print(expert_senses[s])
            
            if mot in dict_of_words[expert_senses[s]]:  # if that word exists in the list of expert words for that sense
                
                #print(s,dict_of_words[expert_senses[s]])
                #print(k_words_with_prob[k][mot])
                
                for word in list_of_all_words:  # this help getting a key for a dictionary later on
                    if mot == word:
                        match_weighted = float((k_words_with_prob[k][mot]))*word_weight[word] #this dictionary cfr comment on line 24
                        # word_weight[word] is already "1/number_of_expert_senses_assigned_to_this_word"
                        
                        #print("sense",expert_senses[s],"word",word,"match_weighted",match_weighted)
                        
                        #print(k,s,conf[k,s])

                        
                        # To fix? 
                        # The way the code works is that all matches happen number_of_s times
                        # (number_of_s = number of expert senses)
                        # easy fix is to divide the match score by number_of_s
                        
                        conf[k,s] = conf[k,s] + match_weighted/4
                        
                    #else: 
                        #print(word,"has no match for sense",expert_senses[s])
                        #print(word,word_weight[word],"match",k_words_with_prob[k][mot],"match weighted",match_weighted)
                    #print("test1")
                #print("test2")
                
                    #compteur += 1
                
        if (k,s) in conf.keys():
        
            conf[k,s] = conf[k,s]/10  # 10 is hardcoded, that's the number of words the model outputs
            print("k =",k,"\t s =",s,"(= expert sense",expert_senses[s],")\t conf[k,s] =",conf[k,s])
            
            #print(compteur)
            
    #print(k_words_with_prob[k])


number of expert senses s: 4
number of model output senses k: 5


Choose best match for k = 0
k = 0 	 s = 0 (= expert sense mus-1 )	 conf[k,s] = 0.0
k = 0 	 s = 1 (= expert sense mus-4 )	 conf[k,s] = 1.0
k = 0 	 s = 2 (= expert sense mus-3 )	 conf[k,s] = 0.0
k = 0 	 s = 3 (= expert sense mus-2 )	 conf[k,s] = 0.0


Choose best match for k = 1
k = 1 	 s = 0 (= expert sense mus-1 )	 conf[k,s] = 0.0
k = 1 	 s = 1 (= expert sense mus-4 )	 conf[k,s] = 0.0
k = 1 	 s = 2 (= expert sense mus-3 )	 conf[k,s] = 1.0
k = 1 	 s = 3 (= expert sense mus-2 )	 conf[k,s] = 0.0


Choose best match for k = 2
k = 2 	 s = 0 (= expert sense mus-1 )	 conf[k,s] = 0.0
k = 2 	 s = 1 (= expert sense mus-4 )	 conf[k,s] = 0.0
k = 2 	 s = 2 (= expert sense mus-3 )	 conf[k,s] = 0.0
k = 2 	 s = 3 (= expert sense mus-2 )	 conf[k,s] = 1.0


Choose best match for k = 3
k = 3 	 s = 0 (= expert sense mus-1 )	 conf[k,s] = 1.0
k = 3 	 s = 1 (= expert sense mus-4 )	 conf[k,s] = 0.0
k = 3 	 s = 2 (= expert sense mus-3 )	 conf[k,

Barbara's note: "key" and "i" may be different senses; for example, for key=0, this is the first sense in the output, and i=0 is the first sense annotated by the expert. I think what we want here is to try matching all key values with all i values; for each (key, i) pair, we get the conf(key, i), as in pages 5 ff. of the Goals and plan.docx document.

Then, when possible, we can pick the best "i" for each "key"; we haven't yet decided how, but it will probably have to do with the maximum conf value.

Once we have a key-->i mapping, we can calculate precision and recall..

## Q3: How well does the model assign the right words to a given sense of the target word?

In [110]:
# For each k, we use the words given by the expert as unquestionable truth.
# Judging the model's assignment of words to a given sense becomes a question of precision and recall.

# precision is all correct w weighted by their respective probabilities / all w weighted by their probabilities

for each k:
    for each w:
        if w in expert_list:
            w_weight = p*1
            numerator += w_weight
        w_weight = p*1
        denominator += w_weight
    precision = numerator/denominator
    
# recall is all correct w weighted by their respective probabilities / all w assigned to the sense by the expert
for each k:
    for each w:
        if w in expert_list:
            w_weight = p*1
            numerator += w_weight
    denominator = len(expert_list)
    recall = numerator/denominator
    
# f-score can be used as well

for each k:
    f_score = 2 * precision * recall / (precision+recall)

SyntaxError: invalid syntax (<ipython-input-110-f315d04cacf9>, line 6)

In [149]:
# TODO: now that we have prob + word for each k,s pair we can calculate precision and recall 

list_with_ks = list()  # this list stores the k,s matches found above
list_with_ks = ["0,0","0,1","0,2","0,3","1,0","1,1","1,2","1,3","2,0","2,1","2,2","2,3","3,0","3,1","3,2","3,3","4,0","4,1","4,2","4,3"]

## RECALL
for key in list_with_ks:
    numerator_recall = 0
    denominator_precision = 0
    numerator_precision = 0
    for word in k_words_with_prob[int(key[0])]:    
        if word in dict_of_words[expert_senses[int(key[2])]]:    # this is bad but works for now
            w_weight_recall = k_words_with_prob[int(key[0])][word] * 1
            numerator_recall += float(w_weight_recall)
            w_weight_precision = k_words_with_prob[int(key[0])][word] * 1
            numerator_precision += float(w_weight_precision)
    
        denominator_precision += float(w_weight_precision)
    denominator_recall = len(dict_of_words[expert_senses[int(key[2])]])
    
            
    print("For pair ks",key,":")
    print("The RECALL is",numerator_recall,"/",denominator_recall,"=",numerator_recall/denominator_recall)
    print("The PRECISION is",numerator_precision,"/",denominator_precision,"=",numerator_precision/denominator_precision,"\n")
    if (numerator_precision/denominator_precision)+(numerator_recall/denominator_recall) != 0:
        print("The F-SCORE is", (2*(numerator_precision/denominator_precision)*(numerator_recall/denominator_recall)/((numerator_precision/denominator_precision)+(numerator_recall/denominator_recall))),"\n")
    else:
        print("No F-SCORE, can't divide by 0\n\n")

For pair ks 0,0 :
The RECALL is 0 / 17 = 0.0
The PRECISION is 0 / 10.0 = 0.0 

No F-SCORE, can't divide by 0


For pair ks 0,1 :
The RECALL is 10.0 / 16 = 0.625
The PRECISION is 10.0 / 10.0 = 1.0 

The F-SCORE is 0.7692307692307693 

For pair ks 0,2 :
The RECALL is 0 / 17 = 0.0
The PRECISION is 0 / 10.0 = 0.0 

No F-SCORE, can't divide by 0


For pair ks 0,3 :
The RECALL is 0 / 17 = 0.0
The PRECISION is 0 / 10.0 = 0.0 

No F-SCORE, can't divide by 0


For pair ks 1,0 :
The RECALL is 0 / 17 = 0.0
The PRECISION is 0 / 10.0 = 0.0 

No F-SCORE, can't divide by 0


For pair ks 1,1 :
The RECALL is 0 / 16 = 0.0
The PRECISION is 0 / 10.0 = 0.0 

No F-SCORE, can't divide by 0


For pair ks 1,2 :
The RECALL is 10.0 / 17 = 0.5882352941176471
The PRECISION is 10.0 / 10.0 = 1.0 

The F-SCORE is 0.7407407407407407 

For pair ks 1,3 :
The RECALL is 0 / 17 = 0.0
The PRECISION is 0 / 10.0 = 0.0 

No F-SCORE, can't divide by 0


For pair ks 2,0 :
The RECALL is 0 / 17 = 0.0
The PRECISION is 0 / 10.0 = 0.

In [59]:
print(k_words_with_prob)

{0: {'28355': '0.088', '69419': '0.069', '57460': '0.056', '114587': '0.042', '42071': '0.041', '35267': '0.035', '51647': '0.035', '64448': '0.023', '45980': '0.018', '53826': '0.017'}, 1: {'79223': '0.063', '92927': '0.057', '46574': '0.038', '67660': '0.038', '103085': '0.036', '86112': '0.030', '101982': '0.029', '75808': '0.026', '68539': '0.024', '54607': '0.024'}, 2: {'62258': '0.080', '64586': '0.052', '58271': '0.048', 'nlsj86871': '0.041', '101851': '0.039', 'nlsj183': '0.037', '75653': '0.031', 'nlsj59923': '0.030', '70105': '0.026', '82758': '0.024'}, 3: {'102000': '0.035', '70495': '0.035', '7182': '0.029', '70958': '0.024', '49589': '0.021', '83174': '0.021', '106676': '0.019', '22209': '0.018', '93388': '0.017', '16132': '0.016'}, 4: {'nlsj5634': '0.039', '61925': '0.037', '12620': '0.030', '69419': '0.027', '104421': '0.027', '91085': '0.024', '71308': '0.022', '115748': '0.021', '19641': '0.020', '5390': '0.019'}}


In [88]:
print(expert_senses)

['mus-1', 'mus-4', 'w', 'mus-2']


In [130]:
for i in expert_senses:
    print(i,dict_of_words[i])

mus-1 {'mus1h', 'mus1q', 'mus1e', 'mus1l', 'mus1n', 'mus1c', 'mus1k', 'mus1o', 'mus1a', 'mus1g', 'mus1f', 'mus1i', 'mus1j', 'mus1b', 'mus1m', 'mus1p', 'mus1d'}
mus-4 {'66761', '75005', '13942', '13813', '104311', '63873', 'mus4c', '13315', '38472', 'mus4b', 'mus4e', '101851', 'mus4a', '73672', '6898', 'mus4d'}
mus-3 {'mus3m', 'mus3b', 'mus3j', 'mus3c', 'mus3i', 'mus3n', 'mus3h', 'mus3f', 'mus3d', 'mus3p', 'mus3a', 'mus3o', 'mus3k', 'mus3q', 'mus3g', 'mus3e', 'mus3l'}
mus-2 {'mus2o', 'mus2h', 'mus2g', 'mus2i', 'mus2l', 'mus2n', 'mus2p', 'mus2q', 'mus2j', 'mus2f', 'mus2m', 'mus2k', 'mus2d', 'mus2e', 'mus2c', 'mus2a', 'mus2b'}
